# Direct Fidelity Estimation example

In [1]:
from pyquil.paulis import ID
from pyquil.gates import X, MEASURE, H, CZ
from pyquil import Program, get_qc
from pyquil.api import get_benchmarker
from forest.benchmarking.direct_fidelity_estimation import ( generate_exhaustive_state_dfe_experiment, 
                                                             generate_exhaustive_process_dfe_experiment,
                                                             acquire_dfe_data,
                                                             estimate_dfe )

In [2]:
# noiseless QVM
qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=False)

# noisy QVM
noisy_qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=True)

bm = get_benchmarker()

### State prep (thing we will do DFE on)

In [3]:
p = Program()
prep_prog = p.inst(CZ(0,1))
print(prep_prog)

CZ 0 1



### Get things required for a DFE experiment ... aka experiment object

Namely return a namedtuple consisiting of 
- pauli_in, 
- prog, 
- pauli_out.

In [4]:
# state dfe
state_exp = generate_exhaustive_state_dfe_experiment(prep_prog,[0,1],bm)

# process dfe
process_exp = generate_exhaustive_process_dfe_experiment(prep_prog,[0,1],bm)

In [5]:
print(process_exp.__doc__)

Experiments to obtain an estimate of fidelity via DFE


In [6]:
process_exp

DFEExperiment(experiments=<pyquil.operator_estimation.TomographyExperiment object at 0x10f82e390>, kind='exhaustive, process')

### Acquire DFE data (noiseless)

Returns dfe data 'data' and calibration data 'cal' namedtuple separately

In [7]:
data = acquire_dfe_data(qvm, process_exp)
print("================================")
print(data.pauli_point_est)
print("================================")
print(data.cal_point_est)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Acquire DFE data (noisy QVM)

In [8]:
n_data = acquire_dfe_data(noisy_qvm, process_exp)
print("================================")
print(n_data.pauli_point_est)
print("================================")
print(n_data.cal_point_est)

[0.99103484 0.99817898 0.99413116 1.00769428 1.00749149 0.9909788
 0.99023136 0.99297607 0.9882021  0.97099622 0.97657841 0.9905448
 0.97558791 0.98027664 0.97430679 1.00751101 0.97747632 0.98002049
 0.98691926 0.9909788  0.97964967 0.98550349 0.98370257 0.97302799
 0.99126413 0.9947644  0.99773191 1.00206239 1.00723701 0.97693491
 0.99128205 0.9828117  0.97671294 0.98652896 1.00090498 0.98954545
 0.98145216 1.00686185 0.9859665  1.00076375 0.98582278 0.99325084
 1.0059415  0.97864209 0.98630137 0.99748801 1.01809721 0.9989648 ]
[0.7808 0.7688 0.7838 0.7798 0.881  0.8868 0.778  0.7688 0.7798 0.793
 0.7856 0.8884 0.893  0.7808 0.7862 0.7722 0.7814 0.7808 0.8868 0.8868
 0.7764 0.7726 0.7854 0.786  0.7784 0.8786 0.8818 0.7758 0.7738 0.7804
 0.78   0.7796 0.8932 0.8908 0.884  0.88   0.8842 0.8744 0.8836 0.7856
 0.79   0.889  0.8752 0.8896 0.8906 0.8758 0.7736 0.7728]


In [9]:
print(n_data.__doc__)

Experimental data from a DFE experiment


### Estimate fidelity (noiseless QVM)

In [11]:
est = estimate_dfe(data)
print("==================================================")
print('Fidelity point estimate is', est.fid_point_est)
print('The variance of Fidelity point estimate is', est.fid_std_err)
print("==================================================")

Fidelity point estimate is 1.0
The variance of Fidelity point estimate is 0.0


### Estimate fidelity (noisy QVM)

In [13]:
nest = estimate_dfe(n_data)
print(nest)
print("==================================================")
print('Fidelity point estimate is', nest.fid_point_est)
print('The variance of Fidelity point estimate is', nest.fid_std_err)
print("==================================================")

DFEEstimate(dimension=4, qubits=[0, 1], fid_point_est=0.9921888856346573, fid_std_err=0.0011514285309063522)
Fidelity point estimate is 0.9921888856346573
The variance of Fidelity point estimate is 0.0011514285309063522
